In [21]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
from qiskit.aqua.components.optimizers import COBYLA

## Optimization of Single Qubit Variational From
Example from: https://qiskit.org/textbook/ch-applications/vqe-molecules.html

"We will now use the simple single qubit variational form to solve a problem similar to ground state energy estimation. Specifically, we are given a random probability vector  $\overrightarrow{x}$ and wish to determine a possible parameterization for our single qubit variational form such that it outputs a probability distribution that is close to $\overrightarrow{x}$ (where closeness is defined in terms of the Manhattan distance between the two probability vectors)."

### Define Target Distribution
we are measuring the probability of the 0 state and the 1 state, so each should be approximately 50/50. Therefore our target distribution is a length two vector with roughly 50% for each component because each number is chosen randomly from 0 to 1. 

In [22]:
np.random.seed(999999)
target_distr = np.random.rand(2)
target_distr /= sum(target_distr)
target_distr

array([0.51357006, 0.48642994])

### Function for Creating Variational Form
It's a circuit with 1 qubit, where the first qubit is acted on by a U3 gate with some input set of parameters.

In [23]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
def get_var_form(params):
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name='c')
    qc = QuantumCircuit(qr, cr)
    qc.u3(params[0], params[1], params[2], qr[0])
    #print(qc)
    qc.measure(qr, cr[0])
    #print(qc)
    return qc

### Specify Objective Function 
We define the objective function, which is the calculation of the quantity of interest, here it's just a cost function is the sum of the absolute value of the difference between the target distribution and the output distribution of the objective function. 

In [24]:
from qiskit import Aer, execute
backend = Aer.get_backend("qasm_simulator")
num_shots = 100

In [25]:
def get_probability_distribution(counts):
    output_distr = [v/num_shots for v in counts.values()]
    if len(output_distr) == 1:
        output_distr.append(0)
    return output_distr

In [26]:
def objective_function(params):
    # Obtain a quantum circuit instance from the parameters
    qc = get_var_form(params)
    #print(qc)
    result = execute(qc, backend, shots=num_shots).result()
    #print(dir(result))
    #print(result.get_counts(qc))
    output_distr = get_probability_distribution(result.get_counts(qc))
    print(output_distr)
    cost = sum([np.abs(output_distr[i] - target_distr[i]) for i in range(2)])
    return cost


In [27]:
params = np.zeros(3)
params[0] = 1.46209037
params[1] = 1.564515
params[2] = 0.34435917
#array([1.46209037, 1.564515  , 0.34435917])
test = objective_function(params)
test

[0.43, 0.57]


0.1671401187388391

In [28]:
# Initialize the COBYLA optimizer
optimizer = COBYLA(maxiter=500, tol=0.001)

In [29]:
params = np.random.rand(3)
ret = optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)

[0.12, 0.88]
[0.51, 0.49]
[0.57, 0.43]
[0.53, 0.47]
[0.98, 0.02]
[0.81, 0.19]
[0.44, 0.56]
[0.57, 0.43]
[0.7, 0.3]
[0.56, 0.44]
[0.67, 0.33]
[0.54, 0.46]
[0.48, 0.52]
[0.63, 0.37]
[0.51, 0.49]
[0.56, 0.44]
[0.65, 0.35]
[0.61, 0.39]
[0.59, 0.41]
[0.45, 0.55]
[0.58, 0.42]
[0.55, 0.45]
[0.61, 0.39]
[0.56, 0.44]
[0.56, 0.44]
[0.55, 0.45]


In [110]:
ret[0]
ret[1]
ret[2]

array([1.57432471, 0.24493779, 0.54862615])

0.00714011873883913

31

`ret[0]` is a 1D numpy.ndarray containing the optimal parameters

`ret[1]` is a float with the objective function value (cost in this case)

`ret[2]` number of objective function calls made if available

And now that we have our optimized parameters, we can calculate how close the measurement of the variational form with these parameters is to the target distribution. 

In [112]:
qc = get_var_form(ret[0])
print(qc)

        ┌────────────────────────────┐┌─┐
q_0: |0>┤ U3(1.5743,0.24494,0.54863) ├┤M├
        └────────────────────────────┘└╥┘
 c_0: 0 ═══════════════════════════════╩═
                                         


In [114]:
counts = execute(qc, backend, shots=num_shots).result().get_counts(qc)
output_distr = get_probability_distribution(counts)

print("Target Distribution:", target_distr)
print("Obtained Distribution:", output_distr)
print("Output Error (Manhattan Distance):", ret[1])
print("Parameters Found:", ret[0])

Target Distribution: [0.51357006 0.48642994]
Obtained Distribution: [0.49, 0.51]
Output Error (Manhattan Distance): 0.00714011873883913
Parameters Found: [1.57432471 0.24493779 0.54862615]


## Domain Agnostic Variational Forms
A domain agnostic variational form is a 'heuristic circuit' where the gates are layered such that they give a good approximation to a wide range of states. Qiskit Aqua has three such types:
* Ry
* RyRx
* SwapRz

Ry, and RyRz accept three inputs that cover the basic parameters:

* number of qubits
* depth setting (number of times to repeat a set pattern of 1-qubit gates and CX gates)
* entanglement setting (how CX gates connect qubits, i.e. linear/nearest-neighbor or full/all-to-all)

In [1]:
from qiskit.aqua.components.variational_forms import RYRZ
entanglements = ["linear", "full"]
for entanglement in entanglements:
    form = RYRZ(num_qubits=4, depth=1, entanglement=entanglement)
    if entanglement == "linear":
        print("=============Linear Entanglement:=============")
    else:
        print("=============Full Entanglement:=============")
    # We initialize all parameters to 0 for this demonstration
    print(form.construct_circuit([0] * form.num_parameters).draw(fold=100))
    print()

=============Linear Entanglement:=============
        ┌───────────┐┌───────┐ ░                                                                 ░ »
q_0: |0>┤ U3(0,0,0) ├┤ U1(0) ├─░───────────────■─────────────────────────────────────────────────░─»
        ├───────────┤├───────┤ ░ ┌──────────┐┌─┴─┐┌──────────┐                                   ░ »
q_1: |0>┤ U3(0,0,0) ├┤ U1(0) ├─░─┤ U2(0,pi) ├┤ X ├┤ U2(0,pi) ├──■────────────────────────────────░─»
        ├───────────┤├───────┤ ░ ├──────────┤└───┘└──────────┘┌─┴─┐┌──────────┐                  ░ »
q_2: |0>┤ U3(0,0,0) ├┤ U1(0) ├─░─┤ U2(0,pi) ├─────────────────┤ X ├┤ U2(0,pi) ├──■───────────────░─»
        ├───────────┤├───────┤ ░ ├──────────┤                 └───┘└──────────┘┌─┴─┐┌──────────┐ ░ »
q_3: |0>┤ U3(0,0,0) ├┤ U1(0) ├─░─┤ U2(0,pi) ├──────────────────────────────────┤ X ├┤ U2(0,pi) ├─░─»
        └───────────┘└───────┘ ░ └──────────┘                                  └───┘└──────────┘ ░ »
«     ┌───────────┐┌───────┐ ░ 
«q_0: ┤ U3(0

## VQE Implementation in Qiskit

In [30]:
from qiskit.aqua.algorithms import VQE, ExactEigensolver
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

In [31]:
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.aqua.components.variational_forms import RYRZ
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.aqua.operators import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel 

In [43]:
def get_qubit_op(dist):
    freeze_list = [0]
    remove_list = [-3, -2]
    driver = PySCFDriver(atom="Li .0 .0 .0; H .0 .0 " + str(dist), unit=UnitsType.ANGSTROM, 
                         charge=0, spin=0, basis='sto3g')
    molecule = driver.run()
    
    ## create lists of orbitals to either freeze or remove when the fermionic operator is constructed
    num_orbitals = molecule.num_orbitals
    remove_list = [x % num_orbitals for x in remove_list]
    freeze_list = [x % num_orbitals for x in freeze_list]
    remove_list = [x - len(freeze_list) for x in remove_list]
    remove_list += [x + num_orbitals - len(freeze_list)  for x in remove_list]
    freeze_list += [x + num_orbitals for x in freeze_list]
    
    ## create fermionic operator with frozen and elimnated modes
    ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
    ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
    ferOp = ferOp.fermion_mode_elimination(remove_list)    
        
    ## calculate number of particles and spin orbitals 
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    num_spin_orbitals -= len(freeze_list)
    num_spin_orbitals -= len(remove_list)
    num_particles -= len(freeze_list)
   
    qubitOp = ferOp.mapping(map_type='jordan_wigner', threshold=0.00000001)
    print(dir(qubitOp))
    print(qubitOp.basis)
    qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles)
    shift = energy_shift + molecule.nuclear_repulsion_energy
    return qubitOp, num_particles, num_spin_orbitals, shift
    
    #print(dir(molecule))
    
    #print("nuclear_repulsion_energy = ", repulsion_energy)
    #print("num_particles =", num_particles)
    #print("num_orbitals = ", num_spin_orbitals)
    #print("remove_list = ", remove_list)
    #print("freeze_list = ", freeze_list)
    #print("len(freeze_list) = ", len(freeze_list))
    #print("ferOp = ", ferOp)
    #print("energy_shift = ", energy_shift)
    #print(dir(qubitOp)) 
dist = 0.5
qubitOp, num_particles, num_spin_orbitals, shift = get_qubit_op(dist)


['__abstractmethods__', '__add__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__iadd__', '__init__', '__init_subclass__', '__isub__', '__le__', '__lt__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__sub__', '__subclasshook__', '__weakref__', '_abc_impl', '_add_or_sub', '_atol', '_basis', '_name', '_paulis', '_paulis_table', '_routine_compute_mean_and_var', '_scaling_weight', '_z2_symmetries', 'add', 'anticommute_with', 'atol', 'basis', 'chop', 'commute_with', 'construct_evaluation_circuit', 'copy', 'evaluate_with_result', 'evaluate_with_statevector', 'evaluation_instruction', 'evolve', 'evolve_instruction', 'from_dict', 'from_file', 'from_list', 'is_empty', 'multiply', 'name', 'num_qubits', 'paulis', 'print_details', 'reorder_paulis', 'rounding', 'simplify', 'sub', 'to_d

num_orbitals =  6


In [ ]:
backend = BasicAer.get_backend("statevector_simulator")
#distances = np.arange(0.5, 4.0, 0.1)
distances = 0.5
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=5)

for dist in distances:
    qubitOp, num_particles, num_spin_orbitals, shift = get_qubit_op(dist)
    result = ExactEigensolver(qubitOp).run()
    exact_energies.append(result['energy'] + shift)
    initial_state = HartreeFock(qubitOp.num_qubits,
                                num_spin_orbitals,
                                num_particles,
                                'parity') 
    var_form = UCCSD(qubitOp.num_qubits,
                     depth=1,
                     num_orbitals=num_spin_orbitals,
                     num_particles=num_particles,
                     initial_state=initial_state,
                     qubit_mapping='parity')
    
    vqe = VQE(qubitOp, var_form, optimizer)
    results = vqe.run(backend)['energy'] + shift
    vqe_energies.append(results)
    print("Interatomic Distance:", np.round(dist, 2), "VQE Result:", results, "Exact Energy:", exact_energies[-1])
    
print("All energies have been calculated")